In [ ]:
using Pkg
Pkg.add("SymPy")

    Updating registry at `C:\Users\earth\.julia\registries\General.toml`
   Resolving package versions...
   Installed SymPy ───────── v2.3.3
   Installed TermInterface ─ v2.0.0
   Installed CommonEq ────── v0.2.1
   Installed SymPyCore ───── v0.3.0
    Updating `C:\Users\earth\.julia\environments\v1.11\Project.toml`
  [24249f21] + SymPy v2.3.3
    Updating `C:\Users\earth\.julia\environments\v1.11\Manifest.toml`
  [3709ef60] + CommonEq v0.2.1
  [24249f21] + SymPy v2.3.3
  [458b697b] + SymPyCore v0.3.0
  [8ea1fca8] + TermInterface v2.0.0
Precompiling project...
  13331.7 ms  ✓ CommonEq
   9754.4 ms  ✓ TermInterface
   3666.1 ms  ✓ SymPyCore
   3922.2 ms  ✓ SymPy
  38520.7 ms  ✓ Roots → RootsSymPyExt
  5 dependencies successfully precompiled in 76 seconds. 346 already precompiled.
  1 dependency had output during precompilation:
┌ Roots → RootsSymPyExt
│  [ Info: Installing sympy via the Conda sympy package...
│  [ Info: Running `conda install -y sympy` in root environment
39mChannels:


In [9]:
Pkg.add("Latexify")

   Resolving package versions...
   Installed Latexify ─ v0.16.7
    Updating `C:\Users\earth\.julia\environments\v1.11\Project.toml`
  [23fbe1c1] + Latexify v0.16.7
    Updating `C:\Users\earth\.julia\environments\v1.11\Manifest.toml`
  [23fbe1c1] ↑ Latexify v0.16.6 ⇒ v0.16.7
Precompiling project...
   4488.6 ms  ✓ Latexify
   1865.9 ms  ✓ Latexify → SparseArraysExt
   1837.7 ms  ✓ Latexify → DataFramesExt
  3 dependencies successfully precompiled in 10 seconds. 348 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version. Otherwise, loading dependents of this package may trigger further precompilation to work with the unexpected version.


In [25]:
# Loading packages
using SymPy, LinearAlgebra,Latexify

In [13]:
#Define Symbols
@syms p q beta_l beta_h

#Define denominators
den1 = q*(1-p) + (1-q)*p # Denominator for Pll, Plh
den2 = q*p + (1 - q)*(1 - p) # Denominator for Phl, Phh

#Define each term
# Pll
term1_Pll = (q^2 * (1 - p)) / den1 * (beta_l / beta_h)
term2_Pll = (p * (1 - q)^2) / den1
Pll =term1_Pll + term2_Pll

# Plh
term1_Plh = ((1 - q) * p * q) / den1
term2_Plh = (q * (1 - q) * (1 - p)) / den1 * (beta_l / beta_h)
Plh = term1_Plh + term2_Plh

# Phl
term1_Phl = (q * p * (1 - q)) / den2
term2_Phl = ((1 - q) * (1 - p) * q) / den2 * (beta_l / beta_h)
Phl = term1_Phl + term2_Phl

# Phh
term1_Phh = (p * q^2) / den2
term2_Phh = ((1 - p) * (1 - q)^2) / den2 * (beta_l / beta_h)
Phh = term1_Phh + term2_Phh

# Build the full Γ matrix
Gamma = SymPy.Matrix([
    [Pll Plh];
    [Phl Phh]
])

# Display the matrix
println("Gamma matrix:")
display(Gamma)

2×2 Matrix{Sym{PyCall.PyObject}}:
       p*(1 - q)^2/(p*(1 - q) + q*(1 - p)) + beta_l*q^2*(1 - p)/(beta_h*(p*(1 - q) + q*(1 - p)))  …  p*q*(1 - q)/(p*(1 - q) + q*(1 - p)) + beta_l*q*(1 - p)*(1 - q)/(beta_h*(p*(1 - q) + q*(1 - p)))
 p*q*(1 - q)/(p*q + (1 - p)*(1 - q)) + beta_l*q*(1 - p)*(1 - q)/(beta_h*(p*q + (1 - p)*(1 - q)))           p*q^2/(p*q + (1 - p)*(1 - q)) + beta_l*(1 - p)*(1 - q)^2/(beta_h*(p*q + (1 - p)*(1 - q)))

Gamma matrix:


In [18]:
#Set p=0.5
Gamma_at_p_half = subs.(Gamma, p => Sym(1)//2)
Gamma_at_p_half = simplify(Gamma_at_p_half)
display(Gamma_at_p_half)

2×2 Matrix{Sym{PyCall.PyObject}}:
       (1 - q)^2 + beta_l*q^2/beta_h  q*(1 - q) + beta_l*q*(1 - q)/beta_h
 q*(1 - q) + beta_l*q*(1 - q)/beta_h        q^2 + beta_l*(1 - q)^2/beta_h

In [42]:
#Comput diagonalization of the matrix
eigen_data = eigen(Gamma_at_p_half)

A = eigen_data.vectors
A = simplify(A)
A_values = eigen_data.values
A_values = simplify(A_values)

2-element Vector{Sym{PyCall.PyObject}}:
 (beta_h + beta_l)*(2*q^2 - 2*q + 1)/(2*beta_h) - sqrt(4*beta_h^2*q^4 - 8*beta_h^2*q^3 + 8*beta_h^2*q^2 - 4*beta_h^2*q + beta_h^2 + 8*beta_h*beta_l*q^4 - 16*beta_h*beta_l*q^3 + 8*beta_h*beta_l*q - 2*beta_h*beta_l + 4*beta_l^2*q^4 - 8*beta_l^2*q^3 + 8*beta_l^2*q^2 - 4*beta_l^2*q + beta_l^2)/(2*beta_h)
 (beta_h + beta_l)*(2*q^2 - 2*q + 1)/(2*beta_h) + sqrt(4*beta_h^2*q^4 - 8*beta_h^2*q^3 + 8*beta_h^2*q^2 - 4*beta_h^2*q + beta_h^2 + 8*beta_h*beta_l*q^4 - 16*beta_h*beta_l*q^3 + 8*beta_h*beta_l*q - 2*beta_h*beta_l + 4*beta_l^2*q^4 - 8*beta_l^2*q^3 + 8*beta_l^2*q^2 - 4*beta_l^2*q + beta_l^2)/(2*beta_h)

In [22]:
#Case2: beta_l=0.3 beta_h=0.6
# Create a substitution dictionary
subs_dict_2 = Dict(
    p => Sym(1)//2,
    beta_l => 0.3,
    beta_h => 0.6
)
Gamma3 = subs.(Gamma, Ref(subs_dict_2))
Gamma3 = simplify(Gamma3)
display(Gamma3)

2×2 Matrix{Sym{PyCall.PyObject}}:
 0.5*q^2 + (1 - q)^2        1.5*q*(1 - q)
       1.5*q*(1 - q)  q^2 + 0.5*(1 - q)^2

In [ ]:
eigen_data_3 = eigen(Gamma3)
A_2 = eigen_data_3.vectors

2×2 Matrix{Sym{PyCall.PyObject}}:
 (3.0*q^2 - 2.0*q + 1.0)/(3.0*q^2 - 3.0*q) - 2.0*(1.5*q^2 - 1.5*q - 2.12132034355964*(0.5*q^4 - q^3 + 0.555555555555556*q^2 - 0.0555555555555556*q + 0.0138888888888889)^0.5 + 0.75)/(3.0*q^2 - 3.0*q)  …  (3.0*q^2 - 2.0*q + 1.0)/(3.0*q^2 - 3.0*q) - 2.0*(1.5*q^2 - 1.5*q + 2.12132034355964*(0.5*q^4 - q^3 + 0.555555555555556*q^2 - 0.0555555555555556*q + 0.0138888888888889)^0.5 + 0.75)/(3.0*q^2 - 3.0*q)
                                                                                                                                                                                      1.00000000000000                                                                                                                                                                                          1.00000000000000

In [36]:
A_2_values = eigen_data_3.values

2-element Vector{Sym{PyCall.PyObject}}:
 1.5*q^2 - 1.5*q - 2.12132034355964*(0.5*q^4 - q^3 + 0.555555555555556*q^2 - 0.0555555555555556*q + 0.0138888888888889)^0.5 + 0.75
 1.5*q^2 - 1.5*q + 2.12132034355964*(0.5*q^4 - q^3 + 0.555555555555556*q^2 - 0.0555555555555556*q + 0.0138888888888889)^0.5 + 0.75

In [27]:
using SymPy: latex

In [ ]:
function export_matrix_latex(matrix, filename)
    open(filename, "w") do f
        write(f, "\\[\n\\begin{bmatrix}\n")
        for i in 1:size(matrix,1)
            row_entries = [string(matrix[i,j]) for j in 1:size(matrix,2)]
            row_latex = join(row_entries, " & ")
            write(f, row_latex * "\\\\\n")
        end
        write(f, "\\end{bmatrix}\n\\]\n")
    end
end

# Then call it:
export_matrix_latex(A_2, "A_output.tex")

17

In [37]:
export_matrix_latex(A_2_values, "A_2_values_output.tex")

17

In [44]:
A_2_inverse = inv(A_2)
export_matrix_latex(A_2_inverse, "A_2_inverse_output.tex")

17

In [43]:
export_matrix_latex(A, "A_output.tex")
export_matrix_latex(A_values, "A_values_output.tex")

17